<a href="https://colab.research.google.com/github/bjungweapon/mjc.ai.ml/blob/BDU/BDU_sklearn_softmax_animation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import make_classification

# (1) 데이터 생성 (3클래스)
X, y = make_classification(
    n_samples=300,
    n_features=2,
    n_informative=2,
    n_redundant=0,
    n_classes=3,
    n_clusters_per_class=1,
    random_state=42
)

# (2) 모델 생성
model = LogisticRegression(multi_class='multinomial', solver='lbfgs')
model.fit(X, y)

# (3) 예측 함수
def predict(X_grid):
    return model.predict(X_grid)


In [ ]:
# (4) 결정 경계 시각화
x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
xx, yy = np.meshgrid(np.linspace(x_min, x_max, 300),
                     np.linspace(y_min, y_max, 300))
grid = np.c_[xx.ravel(), yy.ravel()]
Z = predict(grid)
Z = Z.reshape(xx.shape)

plt.contourf(xx, yy, Z, cmap=plt.cm.Spectral, alpha=0.5)
plt.scatter(X[:, 0], X[:, 1], c=y, edgecolors='k', cmap=plt.cm.Spectral)
plt.title("Softmax Regression Decision Boundary (sklearn)")
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import make_classification

# (1) 데이터 생성
X, y = make_classification(
    n_samples=300,
    n_features=2,
    n_informative=2,
    n_redundant=0,
    n_classes=3,
    n_clusters_per_class=1,
    random_state=42
)

# (2) 모델 초기화
model = LogisticRegression(multi_class='multinomial', solver='lbfgs')

# (3) 시각화용 그리드
x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
xx, yy = np.meshgrid(np.linspace(x_min, x_max, 300),
                     np.linspace(y_min, y_max, 300))
grid = np.c_[xx.ravel(), yy.ravel()]

# (4) 애니메이션 준비
fig, ax = plt.subplots()

def animate(i):
    ax.clear()

    # 점진적으로 학습 데이터 일부만 사용
    num_samples = 10 + i * 5
    if num_samples > len(X):
        num_samples = len(X)

    subset_X = X[:num_samples]
    subset_y = y[:num_samples]

    temp_model = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=200)
    temp_model.fit(subset_X, subset_y)

    Z = temp_model.predict(grid)
    Z = Z.reshape(xx.shape)

    ax.contourf(xx, yy, Z, cmap=plt.cm.Spectral, alpha=0.5)
    ax.scatter(X[:, 0], X[:, 1], c=y, edgecolors='k', cmap=plt.cm.Spectral)
    ax.set_title(f"Softmax Regression Animation - Frame {i}")

ani = animation.FuncAnimation(fig, animate, frames=30, interval=300)

# (5) Colab에서 animation 표시를 위해 JSHTML 모드 사용
from IPython.display import HTML

HTML(ani.to_jshtml())

tatic 시각화: 결정 경계가 한눈에 보인다.

Animation 시각화: 점점 더 많은 데이터로 학습하면서 결정 경계가 변해가는 과정을 직관적으로 이해할 수 있다.

Animation을 만들려면 "부분 학습(subset training)" 개념을 이용해야 하고, sklearn의 기본 fit()은 내부 최적화를 다 끝낸 후 모델을 반환하는 형태이기 때문에 이런 식으로 시뮬레이션하는 거예요.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

# (1) 데이터 생성 (numpy -> torch tensor)
from sklearn.datasets import make_classification

X_np, y_np = make_classification(
    n_samples=300,
    n_features=2,
    n_informative=2,
    n_redundant=0,
    n_classes=3,
    n_clusters_per_class=1,
    random_state=42
)

X = torch.tensor(X_np, dtype=torch.float32)
y = torch.tensor(y_np, dtype=torch.long)

# (2) 모델 정의 (Softmax Regression)
class SoftmaxRegression(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(SoftmaxRegression, self).__init__()
        self.linear = nn.Linear(input_dim, output_dim)

    def forward(self, x):
        return self.linear(x)  # CrossEntropyLoss 내부에 softmax 포함

model = SoftmaxRegression(input_dim=2, output_dim=3)

# (3) 손실 함수와 최적화 방법
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)


In [ ]:
# 시각화용 그리드 생성
x_min, x_max = X_np[:, 0].min() - 1, X_np[:, 0].max() + 1
y_min, y_max = X_np[:, 1].min() - 1, X_np[:, 1].max() + 1
xx, yy = np.meshgrid(np.linspace(x_min, x_max, 300),
                     np.linspace(y_min, y_max, 300))
grid = np.c_[xx.ravel(), yy.ravel()]
grid_torch = torch.tensor(grid, dtype=torch.float32)


In [ ]:
# (4) Animation 준비
fig, ax = plt.subplots()

epochs = 200  # 총 학습 횟수

def animate(epoch):
    ax.clear()

    # 한 번 학습
    outputs = model(X)
    loss = criterion(outputs, y)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # 결정 경계 계산
    with torch.no_grad():
        preds = model(grid_torch)
        preds = torch.argmax(preds, axis=1).numpy()
    Z = preds.reshape(xx.shape)

    ax.contourf(xx, yy, Z, cmap=plt.cm.Spectral, alpha=0.5)
    ax.scatter(X_np[:, 0], X_np[:, 1], c=y_np, edgecolors='k', cmap=plt.cm.Spectral)
    ax.set_title(f"Epoch {epoch+1} / Loss: {loss.item():.4f}")

ani = animation.FuncAnimation(fig, animate, frames=epochs, interval=200)

# Colab용 출력
from IPython.display import HTML
HTML(ani.to_jshtml())
